<a href="https://colab.research.google.com/github/figueiredoelizeu/Projeto_Transfer_Learning_Python/blob/main/Treinamento_Ferramentas_2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Passo 1: Preparar o Ambiente
Primeiro, vamos instalar as bibliotecas necessárias e montar seu Google Drive. Isso é importante para salvar seus arquivos e modelos.

In [11]:
# Instala o YOLOv8
!pip install ultralytics

# Monta o Google Drive para acessar seus dados e salvar o projeto
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Criando o arquivo data.yaml
Este arquivo é essencial. Ele informa ao YOLO onde encontrar as imagens e quais são as classes de objetos que você quer detectar. Crie um arquivo chamado data.yaml e salve-o na pasta ferramentas_ausentes/ com o seguinte

In [14]:
# Lista de todas as ferramentas que deveriam estar no quadro.
# ESSA LISTA DEVE SER IDÊNTICA À DO SEU data.yaml!
todas_as_ferramentas = [
    'alicate-universal',
    'chave-fenda',
    'chave-inglesa',
    'chave-philips',
    'jogo-catraca',
    'jogo-chave-fixa-combinada',
    'alicate-corte',
    'alicate-bico',
    'alicate-anel-elastico-reto',
    'alicate-anel-elastico-curva',
    'estilete',
    'chave-estriada',
    'chave T',
    'fita-isolante',
    'chave-T3',
    'chave-T4',
    'chave-T5',
    'chave-T6',
    'chave-T7',
    'chave-T8',
    'chave-boca-estriada',
    'chave-tampa-fosso-ERC',
    'chave-cotoco-fenda',
    'chave-cotoco-philips',
    'chave-isolada-fenda',
    'chave-isolada-philips',
    'jogo-chave-catraca',
    'jogo-chave-catraca',
    'jogo-chave-catraca-belzer',
    'dispositivo-porta-elevador',
    'chave-boca-gedore-36-32',
    'chave-fixa-combinada-24mm',
    'chave-inglesa-12\'\'',
    'chave-inglesa-300mm',
    'chave-estriada-gedore',
    'lima-meia-lua',
    'lima-reta',
    'chave-fixa-combinada-21mm',
    'chave-fixa-combinada-19mm',
    'chave-fixa-combinada-17mm',
    'chave-fixa-combinada-15mm',
    'chave-fixa-combinada-14mm',
    'chave-fixa-combinada-12mm',
    'chave-fixa-combinada-10mm',
    'chave-fixa-combinada-9mm',
    'chave-canhão-18mm',
    'chave-canhão-12mm',
    'chave-canhão-11mm',
    'chave-canhão-6mm',
    'chave-canhão-5mm',
    'conjunto-chave-fenda',
    'chave-T2',
    'alicate-pressao'
]

# Caminho para a imagem que você quer testar
image_path = '/content/drive/MyDrive/modelo treinado ferramentas/painel_ferramentas.jpg'

# Carrega o modelo treinado
# Substitua o caminho se o seu modelo estiver em outro local
model = YOLO('/content/drive/MyDrive/modelo treinado ferramentas/treinamentos/train3/weights/best.pt')


# Faz a detecção na imagem de teste
results = model(image_path)
result = results[0]

# Define o limite de confiança
confidence_threshold = 0.1 # Você pode ajustar esse limite de confiança (ex: 0.1, 0.3, 0.5)

# Cria uma lista vazia para armazenar as ferramentas que o modelo encontrou
ferramentas_detectadas = []

# Itera sobre as detecções do modelo
for box in result.boxes:
    # Obtém o ID da classe (o número) e a confiança
    class_id = int(box.cls[0].cpu().numpy())
    confidence = box.conf[0].cpu().numpy()

    # Se a confiança for alta o suficiente, adiciona a ferramenta à lista
    if confidence > confidence_threshold:
        class_name = model.names[class_id]
        ferramentas_detectadas.append(class_name)

# Converte as listas para um conjunto para facilitar a comparação e remover duplicatas
set_todas = set(todas_as_ferramentas)
set_detectadas = set(ferramentas_detectadas)

# Encontra as ferramentas ausentes fazendo a diferença entre os dois conjuntos
ferramentas_ausentes = list(set_todas - set_detectadas)

# Imprime o resultado final
print(f'Ferramentas detectadas: {set_detectadas}')
print(f'---')
print(f'Ferramentas ausentes: {ferramentas_ausentes}')


image 1/1 /content/drive/MyDrive/modelo treinado ferramentas/painel_ferramentas.jpg: 640x384 (no detections), 7.7ms
Speed: 2.7ms preprocess, 7.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)
Ferramentas detectadas: set()
---
Ferramentas ausentes: ['chave-T3', 'chave T', 'chave-boca-estriada', 'chave-canhão-11mm', 'chave-inglesa-300mm', 'chave-T2', 'chave-T7', 'chave-canhão-5mm', 'chave-inglesa', 'alicate-anel-elastico-reto', 'alicate-bico', 'jogo-chave-fixa-combinada', 'alicate-pressao', 'alicate-anel-elastico-curva', 'chave-fixa-combinada-17mm', 'chave-boca-gedore-36-32', 'chave-isolada-philips', 'lima-meia-lua', 'alicate-corte', 'chave-isolada-fenda', 'dispositivo-porta-elevador', 'chave-cotoco-philips', 'chave-tampa-fosso-ERC', "chave-inglesa-12''", 'chave-fixa-combinada-9mm', 'chave-cotoco-fenda', 'jogo-catraca', 'fita-isolante', 'chave-fixa-combinada-19mm', 'chave-fixa-combinada-24mm', 'chave-T6', 'chave-estriada', 'chave-fixa-combinada-15mm', 'chave-fixa-com

Este é o coração do sistema aonde detecta as ferramentas e compara com as os frames do video , exporta para uma pasta e salva o arquivo.

In [6]:
import cv2
from ultralytics import YOLO
import time
import pandas as pd # Usaremos pandas para um controle melhor

# Carrega o modelo treinado
model = YOLO('/content/drive/MyDrive/modelo treinado ferramentas/treinamentos/train3/weights/best.pt')

# Lista de referência das ferramentas
todas_as_ferramentas = [
    'alicate-universal', 'chave-fenda', 'chave-inglesa', 'chave-philips', 'jogo-catraca',
    'jogo-chave-fixa-combinada', 'alicate-corte', 'alicate-bico', 'alicate-anel-elastico-reto',
    'alicate-anel-elastico-curva', 'estilete', 'chave-estriada', 'chave T', 'fita-isolante',
    'chave-T3', 'chave-T4', 'chave-T5', 'chave-T6', 'chave-T7', 'chave-T8',
    'chave-boca-estriada', 'chave-tampa-fosso-ERC', 'chave-cotoco-fenda', 'chave-cotoco-philips',
    'chave-isolada-fenda', 'chave-isolada-philips', 'jogo-chave-catraca', 'jogo-chave-catraca-belzer',
    'dispositivo-porta-elevador', 'chave-boca-gedore-36-32', 'chave-fixa-combinada-24mm',
    'chave-inglesa-12\'\'', 'chave-inglesa-300mm', 'chave-estriada-gedore', 'lima-meia-lua',
    'lima-reta', 'chave-fixa-combinada-21mm', 'chave-fixa-combinada-19mm', 'chave-fixa-combinada-17mm',
    'chave-fixa-combinada-15mm', 'chave-fixa-combinada-14mm', 'chave-fixa-combinada-12mm',
    'chave-fixa-combinada-10mm', 'chave-fixa-combinada-9mm', 'chave-canhão-18mm',
    'chave-canhão-12mm', 'chave-canhão-11mm', 'chave-canhão-6mm', 'chave-canhão-5mm',
    'conjunto-chave-fenda', 'chave-T2', 'alicate-pressao'
]
set_todas = set(todas_as_ferramentas)

# Caminho para o vídeo que você quer analisar
video_path = '/content/drive/MyDrive/VID-20250725-WA0028.mp4'

# Inicia a captura de vídeo
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Erro: Não foi possível abrir o vídeo.")
    exit()

# Define o intervalo de tempo para a detecção (em segundos)
# Por exemplo, checar a cada 5 segundos
intervalo_deteccao = 5.0
ultimo_tempo_verificacao = 0

# Define o limite de confiança para as detecções
confidence_threshold = 0.5

# Define o caminho para o arquivo Excel
excel_path = '/content/drive/MyDrive/modelo treinado ferramentas/relatorio_ferramentas_ausentes.xlsx'

# Cria um DataFrame vazio para armazenar os resultados
df_resultados = pd.DataFrame(columns=['Timestamp', 'Ferramentas Ausentes'])

while True:
    # Lê um novo frame
    ret, frame = cap.read()
    if not ret:
        break

    tempo_atual = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

    if tempo_atual - ultimo_tempo_verificacao >= intervalo_deteccao:
        print(f"\n--- Verificando quadro no tempo {tempo_atual:.2f}s ---")

        # Realiza a detecção de objetos no frame
        results = model(frame)
        result = results[0]

        ferramentas_detectadas = []
        for box in result.boxes:
            class_id = int(box.cls[0].cpu().numpy())
            confidence = box.conf[0].cpu().numpy()
            if confidence > confidence_threshold:
                class_name = model.names[class_id]
                ferramentas_detectadas.append(class_name)

        set_detectadas = set(ferramentas_detectadas)
        ferramentas_ausentes = list(set_todas - set_detectadas)

        # Adiciona o resultado ao DataFrame
        novo_registro = {
            'Timestamp': pd.to_datetime(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())),
            'Ferramentas Ausentes': ', '.join(ferramentas_ausentes) if ferramentas_ausentes else 'Nenhuma'
        }
        df_resultados = pd.concat([df_resultados, pd.DataFrame([novo_registro])], ignore_index=True)

        # Salva o DataFrame no arquivo Excel a cada verificação
        df_resultados.to_excel(excel_path, index=False)
        print(f'Relatório salvo em: {excel_path}')

        print(f'Ferramentas detectadas: {set_detectadas}')
        print(f'Ferramentas ausentes: {ferramentas_ausentes}')

        ultimo_tempo_verificacao = tempo_atual

    time.sleep(0.01)

# Libera o objeto de captura
cap.release()
print("\nAnálise do vídeo concluída.")
print(f'Relatório final salvo em: {excel_path}')


--- Verificando quadro no tempo 5.02s ---

0: 640x384 (no detections), 9.8ms
Speed: 2.4ms preprocess, 9.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)
Relatório salvo em: /content/drive/MyDrive/modelo treinado ferramentas/relatorio_ferramentas_ausentes.xlsx
Ferramentas detectadas: set()
Ferramentas ausentes: ['jogo-chave-fixa-combinada', 'chave-fixa-combinada-24mm', 'lima-reta', 'chave-fenda', 'conjunto-chave-fenda', 'estilete', 'chave-fixa-combinada-14mm', 'chave-T3', 'chave-fixa-combinada-21mm', 'alicate-anel-elastico-reto', 'chave-tampa-fosso-ERC', 'chave-canhão-6mm', 'dispositivo-porta-elevador', 'chave-canhão-5mm', 'chave-T5', 'chave-fixa-combinada-9mm', 'chave-T4', 'chave-estriada', 'chave-fixa-combinada-12mm', 'chave-isolada-philips', 'alicate-bico', 'chave-inglesa-300mm', 'chave-canhão-18mm', 'chave-fixa-combinada-10mm', "chave-inglesa-12''", 'chave-inglesa', 'chave-T2', 'chave-estriada-gedore', 'jogo-catraca', 'chave-canhão-11mm', 'chave-fixa-combinada-1

/tmp/ipython-input-3054070898.py:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_resultados = pd.concat([df_resultados, pd.DataFrame([novo_registro])], ignore_index=True)



--- Verificando quadro no tempo 10.04s ---

0: 640x384 (no detections), 7.3ms
Speed: 4.2ms preprocess, 7.3ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)
Relatório salvo em: /content/drive/MyDrive/modelo treinado ferramentas/relatorio_ferramentas_ausentes.xlsx
Ferramentas detectadas: set()
Ferramentas ausentes: ['jogo-chave-fixa-combinada', 'chave-fixa-combinada-24mm', 'lima-reta', 'chave-fenda', 'conjunto-chave-fenda', 'estilete', 'chave-fixa-combinada-14mm', 'chave-T3', 'chave-fixa-combinada-21mm', 'alicate-anel-elastico-reto', 'chave-tampa-fosso-ERC', 'chave-canhão-6mm', 'dispositivo-porta-elevador', 'chave-canhão-5mm', 'chave-T5', 'chave-fixa-combinada-9mm', 'chave-T4', 'chave-estriada', 'chave-fixa-combinada-12mm', 'chave-isolada-philips', 'alicate-bico', 'chave-inglesa-300mm', 'chave-canhão-18mm', 'chave-fixa-combinada-10mm', "chave-inglesa-12''", 'chave-inglesa', 'chave-T2', 'chave-estriada-gedore', 'jogo-catraca', 'chave-canhão-11mm', 'chave-fixa-combinada-